# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [3]:
# Your code here
import json
import requests

def get_keys(path):
    with open(path) as f:
        return json.load(f)

save = "/Users/flatironschool/Documents/Clones/week-2/Yelp Lab on learn"

api_key = 'apC6W5V_e5MB6oVHzSFz7CRmKSaVKBDWlNDWrIhZDwwYHOyUs5abXwKLmxSf6M1CExNI4fYPLBKq6xVhuTS0TGygAypfTT9feHgDbi4xnOY3VTb3Vt1LySr1qfY5XXYx'

term = 'Mexican'
location = 'Manhattan NY'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "Wx6lMnEh7dCmX-9PQF1Ifw", "alias": "toloache-50-new-york", "name": "Toloache 50", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/m5yGUnpSxMB0KaxgwwXMQA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/toloache-50-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ", "review_count": 1831, "categories": [{"alias": "mexican", "title": "Mexican"}], "rating": 4.0, "coordinates": {"latitude": 40.762325, "longitude": -73.9855741}, "transactions": ["pickup", "delivery", "restaurant_reservation"], "price": "$$", "location": {"address1": "251 West 50th St", "address2": "", "address3": "", "city": "New York", "zip_code": "10012", "country": "US", "state": "NY", "display_address": ["251 West 50th St", "New York, NY 10012"]}, "phone": "+12125811818", "display_phone": "(212) 581-1818", "distance": 640.3084851021978}, {"id": "6Ct7OeTRn4caY9g

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [4]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [5]:
response.json()['total']

4200

In [7]:
len(response.json()['businesses'])

10

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request

In [8]:
def url_call():
    api_key = 'apC6W5V_e5MB6oVHzSFz7CRmKSaVKBDWlNDWrIhZDwwYHOyUs5abXwKLmxSf6M1CExNI4fYPLBKq6xVhuTS0TGygAypfTT9feHgDbi4xnOY3VTb3Vt1LySr1qfY5XXYx'

    term = 'Mexican'
    location = 'Manhattan NY'
    SEARCH_LIMIT = 50

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
                }
    response = requests.get(url, headers=headers, params=url_params)
    return response.json()['businesses']

In [9]:
def call_repeat():
    list_of_restos = []
    counter = 0
    while len(list_of_restos) < 500:
        list_of_restos.extend(url_call())
        counter += 50
    return list_of_restos

In [10]:
full_list = call_repeat()

In [12]:
len(full_list)

500

In [14]:
import pandas as pd

In [11]:
full_list

[{'id': 'Wx6lMnEh7dCmX-9PQF1Ifw',
  'alias': 'toloache-50-new-york',
  'name': 'Toloache 50',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/m5yGUnpSxMB0KaxgwwXMQA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/toloache-50-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ',
  'review_count': 1831,
  'categories': [{'alias': 'mexican', 'title': 'Mexican'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.762325, 'longitude': -73.9855741},
  'transactions': ['pickup', 'restaurant_reservation', 'delivery'],
  'price': '$$',
  'location': {'address1': '251 West 50th St',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10012',
   'country': 'US',
   'state': 'NY',
   'display_address': ['251 West 50th St', 'New York, NY 10012']},
  'phone': '+12125811818',
  'display_phone': '(212) 581-1818',
  'distance': 640.3084851021978},
 {'id': '6Ct7OeTRn

In [22]:
df = pd.DataFrame.from_dict(full_list)
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,toloache-50-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 40.762325, 'longitude': -73.9855741}",(212) 581-1818,640.308485,Wx6lMnEh7dCmX-9PQF1Ifw,https://s3-media4.fl.yelpcdn.com/bphoto/m5yGUn...,False,"{'address1': '251 West 50th St', 'address2': '...",Toloache 50,+12125811818,$$,4.0,1831,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/toloache-50-new-york?...
1,la-contenta-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.71873, 'longitude': -73.98691}",(212) 432-4180,4666.898117,6Ct7OeTRn4caY9g0swFBsw,https://s3-media2.fl.yelpcdn.com/bphoto/FwZAyP...,False,"{'address1': '102 Norfolk St', 'address2': '',...",La Contenta,+12124324180,$$,4.5,698,"[pickup, delivery]",https://www.yelp.com/biz/la-contenta-new-york?...
2,mamasita-bar-and-grill-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.76733, 'longitude': -73.98945}",(212) 315-9444,1219.358049,xemN4tP9RlK5c-yUT-M71Q,https://s3-media1.fl.yelpcdn.com/bphoto/39y5db...,False,"{'address1': '818 10th Ave', 'address2': '', '...",Mamasita Bar & Grill,+12123159444,$$,4.0,960,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/mamasita-bar-and-gril...
3,calexico-new-york-5,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.77259, 'longitude': -73.95588}",(347) 967-5955,2349.926470,7bD22UAtH56ikWYslOSIQg,https://s3-media2.fl.yelpcdn.com/bphoto/1vz06a...,False,"{'address1': '1491 2nd Ave', 'address2': '', '...",Calexico,+13479675955,$$,4.0,609,"[pickup, delivery]",https://www.yelp.com/biz/calexico-new-york-5?a...
4,tacuba-hells-kitchen-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.76545, 'longitude': -73.98729}",(212) 245-4500,935.207041,ihzI-4yGesoQuAoIocao8g,https://s3-media1.fl.yelpcdn.com/bphoto/Dw6D5L...,False,"{'address1': '802 9th Ave', 'address2': '', 'a...",Tacuba Hell's Kitchen,+12122454500,$$,4.0,607,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/tacuba-hells-kitchen-...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [29]:
import folium
lat =40.7831
long =-73.9712
yelp_map = folium.Map([lat, long])
yelp_map

In [31]:
x = []
y = []
for element in full_list:
    x.append(element['coordinates']['latitude'])
    y.append(element['coordinates']['longitude'])

points = list(zip(x, y))

for p in points:
    lat = p[0]
    long = p[1]
    marker = folium.Marker(location=[lat, long])
    marker.add_to(yelp_map)
yelp_map

In [42]:
n = []
a = []
pr = []
r = []
for element in full_list:
    n.append(element['name'])
    a.append(element['location']['address1'])
    if element.keys() == 'price':
        pr.append(element['price'])
    else:
        pr.append('No price range given')
    if element['rating'] == True:
        r.append(element['rating'])
    else:
        r.append('Not rated yet')

koints = list(zip(n, a, pr, r))

In [33]:
for k in koints:
    Name = k[0]
    Address = k[1]
    price = k[2]
    Rating = k[3]
    popup_text = "Name: {} Address: {} Price: {}, Rating: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(yelp_map)
yelp_map

NameError: name 'base_map' is not defined

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!